In [ ]:
from google.colab import drive
import os

drive.mount("/content/drive")
path = "/content/drive/My Drive/Facial_expression_2/"
os.chdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import keras
import numpy as np 
import cv2
from sklearn.model_selection import train_test_split
from keras.applications import VGG16,VGG19
from keras.layers import Dense, Flatten,Dropout
from keras.models import Model,Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
import pickle


In [ ]:
def save_pickle(obj, file_path):
  with open(file_path, 'wb') as f:
    pickle.dump(obj,f)

def load_pickle(file_path):
  with open(file_path,'rb') as f:
    obj = pickle.load(f)
  return obj

In [ ]:
IMG_SIZE = (224,224)
IMG_SHAPE = (224,224,3)
BATCH_SIZE = 32

In [ ]:
# IMG_SIZE = (48,48)
# IMG_SHAPE = (48,48,3)
# BATCH_SIZE = 32

In [ ]:

# count = 0
# path_train = path + 'Data/train'
# # for file in os.listdir(path_train):
# file = 'sad'
# data = []
# labels = []
# for img in os.listdir(path_train +'/'+file):
#   try:
#     image= cv2.imread(path_train +'/'+file+'/'+img)

#     data.append(image)
#     labels.append(file)
#     print(count )
#     count+=1
#   except:
#     print('error')

# save_pickle(data,'pkl_48/data_train_'+ file +'.pkl')
# save_pickle(labels,'pkl_48/labels_train_' + file + '.pkl')

In [ ]:
# count = 0
# path_val = path + 'Data/validation/'
# data = []
# labels = []
# for file in os.listdir(path_val):
#   for img in os.listdir(path_val +'/'+file):
#     try:
#       image= cv2.imread(path_val +'/'+file+'/'+img)
#       data.append(image)
#       labels.append(file)
#       print(count )
#       count+=1
#     except:
#       print('error')

# save_pickle(data,'pkl_48/data_val.pkl')
# save_pickle(labels,'pkl_48/labels.pkl')

In [ ]:
path_train = path + 'Data/train'
X_train = []
y_train = []
for file in ('angry','fear','happy','neutral','sad','surprise'):
  X = load_pickle('pkl_224/data_train_'+ file +'.pkl')
  y = load_pickle('pkl_224/labels_train_'+ file +'.pkl')
  print(file + str(len(X)) )
  X_train.extend(X)
  y_train.extend(y)

angry3999
fear4103
happy7173
neutral4992
sad4938
surprise3205


In [ ]:
X_val = load_pickle('pkl_224/data_val.pkl')
y_val = load_pickle('pkl_224/labels_val.pkl')

X_val6 = []
y_val6 = []
for num,i in enumerate(y_val):
  if i == 'disgust':
    continue 
  X_val6.append(X_val[num])
  y_val6.append(y_val[num])


In [ ]:
print(len(y_val))
print(len(y_val6))

7070
6959


In [ ]:
for x in X_train:
  x = cv2.dnn.blobFromImage(x, 1.0, IMG_SIZE, [104, 117, 123], False, False)
  mean,std =  x.mean(), x.std()
  x = (x - mean) / std
  
for x in X_val6:
  x = cv2.dnn.blobFromImage(x, 1.0, IMG_SIZE, [104, 117, 123], False, False)
  mean,std =  x.mean(), x.std()
  x = (x - mean) / std

In [ ]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()
encode.fit(y_train)
y_train = encode.transform(y_train)
y_val6 = encode.transform(y_val6)

In [ ]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_val6 = np.asarray(X_val6)
y_val6 = np.asarray(y_val6)

In [ ]:
VGG_model = VGG19(
      include_top = False,
      weights='imagenet',
      input_shape = IMG_SHAPE
  )
model = Sequential()

for layer in VGG_model.layers:
  layer.trainable = False
  model.add(layer)

model.add(Flatten())
model.add(Dense(2048*2, activation='relu') )
model.add(Dense(2048, activation='relu') )
# model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu') )
model.add(Dense(7, activation='softmax') )


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [ ]:
# aug = ImageDataGenerator(
#     rotation_range=20,
# 		zoom_range=0.15,
# 		width_shift_range=0.2,
# 		height_shift_range=0.2,
# 		shear_range=0.15,
# )

In [ ]:
# from tensorflow.keras.callbacks import ModelCheckpoint
# checkpoint_path = 'checkpoints/'
# epochs = 10

# callbacks = [
#     keras.callbacks.ModelCheckpoint(
#       filepath=checkpoint_path,
#       save_freq='epoch',
#       save_best_only=True,
#       verbose=1
#     ),
# ]
# model.compile(
#     optimizer=keras.optimizers.Adam(1e-3),
#     loss="sparse_categorical_crossentropy",
#     metrics=["accuracy"],
# )


In [ ]:
# model.fit(x =aug.flow(X_train,y_train,batch_size=64),
#           epochs=10,     
#           validation_data=(X_val,y_val ),
#           callbacks=[checkpoint])

In [ ]:
epochs = 20
checkpoint_path = 'checkpoints/'
callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_path,
      save_freq='epoch',
      monitor='val_accuracy',
      save_best_only=True,
      verbose=1
    ),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

model.fit(
    X_train,
    y_train,
    epochs=epochs,
    validation_data=(X_val6,y_val6),
    callbacks=callbacks
)


Epoch 1/20
888/888 [==============================] - ETA: 0s - loss: 3.0529 - accuracy: 0.4890
Epoch 00001: val_accuracy improved from -inf to 0.51631, saving model to checkpoints/
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: checkpoints/assets
888/888 [==============================] - 198s 223ms/step - loss: 3.0529 - accuracy: 0.4890 - val_loss: 1.2689 - val_accuracy: 0.5163
Epoch 2/20
888/888 [==============================] - ETA: 0s - loss: 1.0295 - accuracy: 0.6251
Epoch 00002: val_accuracy improved from 0.51631 to 0.56373, saving model to checkpoints/
INFO:tensorflow:Assets written to: checkpoints/assets
888/888 [==============================] - 197s 222ms/step - loss: 1.0295 - accuracy: 0.6251 - val_loss: 1.2087 - val_accuracy: 0.5637
Epoch 3/20
888/888 [=

In [ ]:
model=keras.models.load_model('checkpoints/')

In [ ]:
model.save('face_expression_59.53.h5')